# Lab 9 - adding a database


In [ ]:
# imports

import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

load_dotenv(override=True)


In [2]:
MODEL = "gpt-4.1-mini"
openai = OpenAI()

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, witty, humorous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Some database functions

In [4]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [5]:
def get_ticket_price(city):
    print(f"TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"${result[0]}" if result else "Not found"

In [ ]:
get_ticket_price("London")

In [7]:
def set_ticket_price(city, price):
    print(f"TOOL CALLED: Setting price for {city} to {price}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [ ]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1400, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [ ]:
get_ticket_price("London")

In [10]:
price_function = {
    "name": "get_ticket_price",
    "description": "This tool retrieves the ticket price of a return ticket to the destination city. Call this whenever you need to know a ticket price.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    }
}

In [ ]:
tools = [{"type": "function", "function": price_function}]
tools

In [12]:
def call_tool_get_ticket_price(city):
    price = get_ticket_price(city)
    price_json = {"destination_city": city, "price": price}
    return json.dumps(price_json)

In [13]:
def handle_tool_call(tool_call):
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    results = call_tool_get_ticket_price(city)
    response = {"role": "tool", "content": results, "tool_call_id": tool_call.id}
    return response

In [14]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        for tool_call in message.tool_calls:
            tool_response = handle_tool_call(tool_call)
            messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [16]:
set_price_function = {
    "name": "set_ticket_price",
    "description": "Use this tool to store the price of a return ticket to the destination city in the database, providng the destination city and the price in US Dollars",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "number",
                "description": "The price of the ticket as a number in USD",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False,
    }
}

In [ ]:
admin_tools = [{"type": "function", "function": set_price_function}]
admin_tools

In [18]:
def call_tool_set_ticket_price(city, price):
    set_ticket_price(city, price)
    return f"Price for {city} set to {price}"

In [19]:
def handle_admin_tool_call(tool_call):
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = arguments.get('price')
    results = call_tool_set_ticket_price(city, price)
    response = {"role": "tool", "content": results, "tool_call_id": tool_call.id}
    return response

In [20]:
admin_system_message = """
You are an admin agent of an Airline System for FlightAI, an Airline.
You manage a database of ticket prices.
You are chatting with an internal administrator at the company, who is able to ask you to set the ticket prices in the database.
As required, use your tool to set ticket prices in your database.
"""


In [21]:
def admin_chat(message, history):
    messages = [{"role": "system", "content": admin_system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=admin_tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        for tool_call in message.tool_calls:
            tool_response = handle_admin_tool_call(tool_call)
            messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=admin_chat, type="messages").launch()